In [9]:
import json,codecs
import urllib.request
import urllib.parse

def getWL(province_name, amphoe_names):
    url = "https://api-v3.thaiwater.net/api/v1/thaiwater30/public/waterlevel_load"
    with urllib.request.urlopen(url) as response:
        data = json.loads(response.read())
    data = data["waterlevel_data"]["data"]
    # data
    print(len(data))
    def locationFilter(item):
        # print(item)
        # return (item["geocode"]["amphoe_name"]["th"] in amphoe_names) and (item["geocode"]["province_name"]["th"] == province_name )
        return (item["geocode"]["province_name"]["th"] == province_name )

    filterData = filter(locationFilter,data)
    filterData = list(filterData)
    print(len(filterData))
    return filterData

def getStations(province_name, amphoe_names):
    url = "https://api-v3.thaiwater.net/api/v1/thaiwater30/public/waterlevel_load"
    with urllib.request.urlopen(url) as response:
        data = json.loads(response.read())
    data = data["waterlevel_data"]["data"]
    # data
    # print(len(data))
    def filterWLFn(item):
        # print(item)
        # return (item["geocode"]["amphoe_name"]["th"] in amphoe_names) and (item["geocode"]["province_name"]["th"] == province_name )
        return (item["geocode"]["province_name"]["th"] == province_name )

    filterWL = filter(filterWLFn,data)
    filterWL = list(filterWL)
    # print(len(filterData))


    def mapStationsFn(item):
        return {
            "id":item["station"]["id"],
            "tele_station_name":item["station"]["tele_station_name"]["th"],
            "tele_station_lat":item["station"]["tele_station_lat"],
            "tele_station_long":item["station"]["tele_station_long"],
            "is_key_station":item["station"]["is_key_station"],
            "tumbon_name":item["geocode"]["tumbon_name"]["th"],
            "amphoe_name":item["geocode"]["amphoe_name"]["th"],
            "province_name":item["geocode"]["province_name"]["th"],
            "river_name":item["river_name"],
        }
    
    stations = list(map(mapStationsFn, filterWL))


    return stations

if __name__ == "__main__":
    wl = getStations(province_name="สระแก้ว", amphoe_names=["คลองหาด","วัฒนานคร","อรัญประเทศ"])
    with codecs.open('data/stations.json', 'w', encoding='utf-8') as f:
        json.dump(wl, f, ensure_ascii=False)
        print("Write file data/stations.json")



Write file data/stations.json


In [9]:
target_years = list(range(2015,2025))
target_years = ",".join(map(str, target_years))
target_years

'2015,2016,2017,2018,2019,2020,2021,2022,2023,2024'

In [10]:
import json,codecs
import urllib.request
import urllib.parse

# อ่านไฟล์ JSON
file_path = 'data/stations.json'

with open(file_path, 'r', encoding='utf-8') as file:
    stations = json.load(file)  # อ่านข้อมูล JSON และแปลงเป็น object ใน Python

target_years = list(range(2015,2025))
target_years = ",".join(map(str, target_years))

for station in stations:
    station_id=station["id"]
    print(station_id)
    url = f"https://api-v3.thaiwater.net/api/v1/thaiwater30/public/waterlevel_graph_year?station_type=tele_waterlevel&station_id={station_id}&year={target_years}"
    with urllib.request.urlopen(url) as response:
        data = json.loads(response.read())
    data["station"] = station
    print(url)
    with codecs.open(f"data/station-{station_id}-[2015-2024].json", 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False)
        print(f"Write file data/station-{station_id}-[2015-2024].json")



192
https://api-v3.thaiwater.net/api/v1/thaiwater30/public/waterlevel_graph_year?station_type=tele_waterlevel&station_id=192&year=2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
Write file data/station-192-[2015-2024].json
191
https://api-v3.thaiwater.net/api/v1/thaiwater30/public/waterlevel_graph_year?station_type=tele_waterlevel&station_id=191&year=2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
Write file data/station-191-[2015-2024].json
2677
https://api-v3.thaiwater.net/api/v1/thaiwater30/public/waterlevel_graph_year?station_type=tele_waterlevel&station_id=2677&year=2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
Write file data/station-2677-[2015-2024].json
2675
https://api-v3.thaiwater.net/api/v1/thaiwater30/public/waterlevel_graph_year?station_type=tele_waterlevel&station_id=2675&year=2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
Write file data/station-2675-[2015-2024].json
1098950
https://api-v3.thaiwater.net/api/v1/thaiwater30/public/waterlevel_graph_year?station_t

In [11]:
import json,codecs
import urllib.request
import urllib.parse
import pandas as pd

# อ่านไฟล์ JSON
file_path = 'data/stations.json'

with open(file_path, 'r', encoding='utf-8') as file:
    stations = json.load(file)  # อ่านข้อมูล JSON และแปลงเป็น object ใน Python

df = pd.DataFrame(stations)
df.to_excel(f"data/stations.xlsx", index=False)  # บันทึกเป็น Excel
print("print excel all stations")
# exit()

big_df = None
for station in stations:
    station_id=station["id"]
    file_path = f"data/station-{station_id}-[2015-2024].json"
    with open(file_path, 'r', encoding='utf-8') as file:
        obj = json.load(file)  # อ่านข้อมูล JSON และแปลงเป็น object ใน Python

    # merge data in 5 years
    data = []
    for year_obj in obj["data"]["graph_data"]:
        data += year_obj["data"]          

    df = pd.DataFrame(data)

    # เพิ่มคอลัมน์ใหม่ด้วยค่าคงที่
    df["station_id"] = station["id"]  # เพิ่มคอลัมน์
    df["tele_station_name"] = station["tele_station_name"]  # เพิ่มคอลัมน์
    df["tele_station_lat"] = station["tele_station_lat"]  # เพิ่มคอลัมน์
    df["tele_station_long"] = station["tele_station_long"]  # เพิ่มคอลัมน์
    df["is_key_station"] = station["is_key_station"]  # เพิ่มคอลัมน์
    df["province_name"] = station["province_name"]  # เพิ่มคอลัมน์
    df["amphoe_name"] = station["amphoe_name"]  # เพิ่มคอลัมน์
    df["tumbon_name"] = station["tumbon_name"]  # เพิ่มคอลัมน์
    df["river_name"] = station["river_name"]  # เพิ่มคอลัมน์

    if big_df is None:
        big_df = df
    else:
        # ใช้ pd.concat() เพื่อเพิ่มข้อมูล
        big_df = pd.concat([big_df, df], ignore_index=True)

    df.to_excel(f"data/station-{station_id}.xlsx", index=False)  # บันทึกเป็น Excel
    print("print excel station")

    print(station["tele_station_name"])

big_df.to_excel("data/output-10y-[2015-2024].xlsx", index=False)  # บันทึกเป็น Excel
print("print big excel")

print excel all stations
print excel station
ตาพระยา
print excel station
เมืองสระแก้ว


C:\Users\User\AppData\Local\Temp\ipykernel_3136\1809660437.py:46: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  big_df = pd.concat([big_df, df], ignore_index=True)


print excel station
บ้านสระขวัญ
print excel station
บ้านเขาฉกรรจ์
print excel station
บ้านแก้ง


C:\Users\User\AppData\Local\Temp\ipykernel_3136\1809660437.py:46: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  big_df = pd.concat([big_df, df], ignore_index=True)


print excel station
คลองพรหมโหด


C:\Users\User\AppData\Local\Temp\ipykernel_3136\1809660437.py:46: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  big_df = pd.concat([big_df, df], ignore_index=True)


print excel station
คลองพระปรง
print big excel
